In [2]:
import os
from thermof.read import read_log, read_thermo
import numpy as np
from wham2d import *
import matplotlib.pyplot as plt
%matplotlib inline

## WHAM 2-D

In [ ]:
# SCANS = ['DC_Cu110', 'HtBDC_Cu110', 'C60_Cu110', 'PCARBORANE_Cu110', 'BtPHD_Cu110', 'DNHD_Cu110', 'PVBA_Cu110', 'TPEE_Cu110', 'VL_Cu110']
SCAN = 'DC_Cu110'
SCANDIR = '/home/kutay/Documents/git/Nanocar/surface-scan'
DATADIR = os.path.join(SCANDIR, 'analysis', 'data', SCAN)
WHAMDIR = os.path.join(SCANDIR, 'analysis', 'wham', 'run2d', SCAN)
TSDIR = os.path.join(SCANDIR, 'analysis', 'wham', 'run2d', SCAN, 'data')
os.makedirs(TSDIR, exist_ok=True)
scanlist = [os.path.join(DATADIR, i) for i in os.listdir(DATADIR)]

In [ ]:
thermo_headers = 'Step Temp Press TotEng E_pair E_mol Fmax Fnorm c_C1[1] c_C1[2] c_C1[3]'
thermo_keys = ['step', 'temp', 'press', 'etotal', 'epair', 'emol', 'fmax', 'fnorm', 'x', 'y', 'z']

### Write 2D WHAM time series files

In [ ]:
K_SPRING = 200                                 # Spring constant in kcal/mol
TIMESTEP = 1                                   # Timstep in femtoseconds
T_CONVERSION = 1e-3                            # femtosecond -> picosecond
T_SKIP = 5                                     # Skip first 5 ps of simulation for WHAM analysis

In [ ]:
start_pos = {}
for simdir in scanlist:
    logfile = os.path.join(simdir, 'log.%s' % SCAN)
    try:
        xi, yi = [int(i) for i in os.path.basename(simdir).split('-')]
        thermo_data = read_log(logfile, headers=thermo_headers)
        thermo = read_thermo(thermo_data, headers=thermo_keys)[0]
        # Write time series files
        timeseriesfile = os.path.join(WHAMDIR, 'data', '%i-%i-2d.dat' % (xi ,yi))
        time = timesteps_to_time(thermo['step'], dt=1, conversion=1e-3)  # femtosecond -> picosecond
        time = timesteps_to_time(thermo['step'][T_SKIP:], dt=TIMESTEP, conversion=T_CONVERSION, shift=int(T_SKIP * TIMESTEP))
        write_timeseries_file(timeseriesfile, time, thermo['x'][T_SKIP:], thermo['y'][T_SKIP:])
        for drx in ['x', 'y']:
            # Record starting position (min energy pos)
            start_pos['%i-%i-%s' % (xi ,yi, drx)] = thermo[drx][0]
    except Exception as e:
        print(e, logfile)

### Write 2D WHAM input files

In [ ]:
nx, ny = 22, 34
tsfiles, mins_x, mins_y = [], [], []
for ypos in range(ny):
    for xpos in range(nx):
        tsfiles.append(os.path.join(WHAMDIR, 'data', '%i-%i-2d.dat' % (xpos ,ypos)))
        mins_x.append(start_pos['%i-%i-x' % (xpos ,ypos)])
        mins_y.append(start_pos['%i-%i-y' % (xpos ,ypos)])

datafile = os.path.join(WHAMDIR, 'wham-2d.in')
spring_k = [K_SPRING] * len(tsfiles)
write_data_file(datafile, tsfiles, mins_x, mins_y, spring_k, spring_k)

### Run 2D WHAM

In [ ]:
hist_min_x, hist_max_x = 21, 26
hist_min_y, hist_max_y = 22, 25.8
bin_size_x, bin_size_y = 0.2, 0.1
num_bins_x = int(np.ceil((hist_max_x - hist_min_x) / bin_size_x))
num_bins_y = int(np.ceil((hist_max_y - hist_min_y) / bin_size_y))
Px, Py = 'Px=0', 'Py=0'
tolerance = 1e-5
temperature = 200
numpad = 0
use_mask = 0
wham_exec = '/home/kutay/Documents/Research/Software/wham/wham-2d/wham-2d'
outfile = os.path.join(WHAMDIR, 'wham-2d.out')
wham_args = [wham_exec, Px, hist_min_x, hist_max_x, num_bins_x, Py, hist_min_y, hist_max_y, num_bins_y, tolerance, temperature, numpad, datafile, outfile, use_mask]

In [ ]:
data = run_wham(wham_args, verbose=False)